<a href="https://colab.research.google.com/github/Mattshanevdberg/ML-FruitPunchAI_BootCamp/blob/main/2_2_Remote_experiments_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 2
For this challenge we will again need to use NGROK to connect to the underlying server, so again, run the code below.

In [2]:
#CODE
#Generate root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc
#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')
#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/get-started/your-authtoken")
import getpass
authtoken = getpass.getpass()




Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:uCOAep7UBcaKG41F4AZlpIRdEEXXCISPqItBkQk8zvo root@acb24215f2b6 (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:YyWITO8Ai++y69a5cv7QYsvtAzXbm01/ugxeV3CP4gE root@acb24215f2b6 (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:k9ZufQ1dNOtZb8ekVs3/d9azuzThhTdvAqDDb6JPVDc root@acb24215f2b6 (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Copy authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
··········


In [4]:
# If you get a HTTPConnectionPool error your still have an open NGROK connection in another Colab, kill it or factory reset that Colab
#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
import requests
#Get public address
url = 'http://localhost:4040/api/tunnels'
print(requests.get(url=url).json()['tunnels'][0]['public_url'])
public_adress = requests.get(url=url).json()['tunnels'][0]['public_url']
port = public_adress.split(':')[2]
server = public_adress.split('//')[1].split(':')[0]
print(f' server: {server}\n port: {port}\n username: root\n password: {password}')

tcp://8.tcp.ngrok.io:17868
 server: 8.tcp.ngrok.io
 port: 17868
 username: root
 password: 6Oog6MfuE0QVAilFPSkU


## Connect to the SSH server
Again you would need to connect to the SSH server with the parameters printed above. However, since this is a completely new server, you would need to add your public key again. Luckily this can be done a bit easier from a notebook like this, simply replace the public_key string below with your public key and run the code block

You can now connect to the server with the parameters above and your private key

In [5]:
import os
public_keys = 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCgTMwiLTS6f/utjtT/lLfGsO8lNC/rkWEdufSblDvw/lEqaj1YssBnHjGoJ6rXGNn6T6guvSemdWne4hJstGDdj61yQ== sako@fruitpunch.ai'
public_key = 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDEDHbX1lmv0uYqJucBewP+xZVHGh9Q1QvHGhqrsgshzouXeTGFJ7thw5S3oxKVFZXUVRlmHgi+U2RCDfm3csOe6r3yRYY1LjT+1lniIYk9+gNe0y85Z2aeGedQsECK54NmLbsAGeNhJhwhJMR4j8DYnWJmEjlQ+bWczmpKNJRHK7jZxvuMOHaP8lDjDi7XPS8gpVQ6SdTOtQtrUdf9prfdU9X/ztJINDqpA1XGsr6JmZDgjrGqDNWIV+U5abwuH+D4ZKc5yzv7+w/CdQPdDOVk4nWFLi+qt3+EG5+YPY5oWIUuQsDjPBMRmzMwskZKhSa5iyyYboLL2UhzxbxFSqhZ9b3puEqrYNFYQZYsPpv5AC3E+KDibHOu0qvwsU6fVlA/7J2r+zzN3mOPsS0YM/aDajg0BvHPsSEntcF/DZe9TOGtuOp5Qan6YYirUwAd/WP4G3Neb597niQAfvKQ8JLDJRzQLkMupCNmDyBzfztMRsZIwLosWdTH6Jd2xgumFnMkEVuk4GZ5n5iB94yd2GcAi2DWcLdk8IX7Bt0OL5a7U1Rsf2MB8t8bnvkyr+zxA3XK6N0Nt6y78UaJpRqiuVoB4qDV286BJVQB6SG/xF30dzMxazL+jNERwCSXVZJjTXciQIcwMLRd8L50juFLYcQ6YpU4aT5wVrJOAqcXxqTipw== vanmat10@gmail.com'
os.makedirs('/root/.ssh', exist_ok=True)
with open('/root/.ssh/authorized_keys', "w") as f:
    f.write(public_key)


In [8]:
print("hello")

hello


## Challenge 2.1: Run remote experiment
Connecting to servers like this also allows you to run python code on them, for example a Machine Learning experiment without having you own laptop overheat on calucationtions. You can use screen like in challenge 1.2 to run them overnight, or run multiple at a time. You can even use this trick to run multiple experiments in paralel in on a Colab server like this. 

1. Clone the following git repro via the SSH connection
https://gitlab.com/fruitpunch/education/events/bootcamp-2021-ai-in-practise/session-2-developer-skills.git
2. Move inside the newly created session-2-developer-skills folder
3. Start screen
4. Create new environment
5. Load environment
6. Install packages from the requirements.txt
7. Run experiment.py and detach screen


Next step, create your own git repro, intialize it with code above, change it to a flask api, commit and push changes lcoally then pull them remotely. Then launch API

In [ ]:
! git clone https://gitlab.com/fruitpunch/education/events/bootcamp-2021-ai-in-practise/session-2-developer-skills.git


Cloning into 'session-2-developer-skills'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [ ]:
! cd session-2-developer-skills && git checkout docker

Branch 'docker' set up to track remote branch 'docker' from 'origin'.
Switched to a new branch 'docker'


In [ ]:
! ls

cloudflared	 ngrok			       sample_data
cloudflared.log  ngrok-stable-linux-amd64.zip  session-2-developer-skills


In [ ]:
! python ./session-2-developer-skills/main.py

 * Serving Flask app "main" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Feb/2022 15:13:39] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [16/Feb/2022 15:13:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Feb/2022 15:13:40] "GET / HTTP/1.1" 405 -
